<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2d947cf1bcb923f153644572ef5863df65ccd809a107b530be360f7f58b2d0ce
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 11:48:16
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 14.59 K (0.11%)
Current PnL: -29.16 L (-18.84%)
CY Booked + Current PnL: -14.87 L (-9.61%)
-------------------
Total profit:  1.22 L
Total loss:  -30.38 L
-------------------
Total Booked + Current PnL: 12.11 L (9.47%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.79 L (67.42%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.01,-17.40,21.18,0.09,17630.0,-17540.0,83237.0,93.17,37.0,M-SC,2.35,253.0,-0.99,0.62,9.36,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.72,39.54,XR,ATH,FINANCE
43,ITC,452.00,-0.22,-1.72,12.37,10.44,24332.0,-3438.0,196700.0,-41.15,45.0,X-LC,1.76,5.0,-0.14,1.47,3.75,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-1.33,-14.02,16.33,0.02,24508.0,-24482.0,150077.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.54,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,0.10,5.01,13.98,19.69,28148.0,9603.0,201345.0,-5.22,46.0,X-MC,2.55,78.0,0.34,1.50,12.15,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,806.63,3.54,-15.53,65.21,39.54,55718.0,-15715.0,85444.0,-33.96,49.0,M-SC,4.61,220.0,-0.28,0.64,20.48,AR,ATH,AUTO
22,DIXON,18931.72,2.72,-18.52,49.22,21.58,56201.0,-25956.0,114183.0,-58.94,24.0,X-MC,7.16,56.0,-0.46,0.85,2.72,X40N,ATH,IT
79,VAIBHAVGBL,521.00,2.58,-25.60,116.09,60.77,157859.0,-46795.0,135980.0,57.02,49.0,H-SC,4.14,159.0,-0.30,1.01,22.46,XR,NTT,JEWELLERY
28,GREENPANEL,537.00,1.82,-37.39,128.62,43.14,147432.0,-68452.0,114626.0,156.46,31.0,M-SC,9.51,240.0,-0.46,0.85,7.82,XY24,NTT,MISC
45,JIOFIN,387.00,1.62,-4.93,30.94,24.48,78641.0,-13192.0,254173.0,-6.53,41.0,X-LC,5.93,36.0,-0.17,1.89,47.33,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-2.09,-4.22,26.48,21.14,57660.0,-9591.0,217748.0,-11.03,39.0,X-MC,4.16,74.0,-0.17,1.62,22.73,XY24,NTT,PAINTS
14,CAMPUS,393.0,-1.75,-10.50,48.92,33.28,73458.0,-17616.0,150159.0,-27.96,40.0,M-SC,2.80,222.0,-0.24,1.12,16.38,XY24,NTT,FOOTWEAR
16,CERA,9475.0,-1.65,-31.13,79.89,23.89,96780.0,-54762.0,121141.0,-33.65,30.0,H-SC,9.75,157.0,-0.57,0.90,4.62,OX40N,NTT,CERAMICS
59,RELAXO,1176.0,-1.47,-47.46,189.91,52.31,144829.0,-68898.0,76262.0,-44.05,43.0,X-SC,6.43,91.0,-0.48,0.57,2.08,X40N,NTT,FOOTWEAR
30,HATHWAY,31.4,-1.40,-30.13,148.03,73.29,112444.0,-32760.0,75960.0,780.43,39.0,H-SC,7.50,172.0,-0.29,0.57,2.43,XR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.13,-7.26,124.18,107.91,170041.0,-10713.0,136931.0,-24.60,37.0,M-SC,12.38,216.0,-0.06,1.02,2.10,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.33,-14.02,16.33,0.02,24508.0,-24482.0,150077.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.54,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.01,-17.40,21.18,0.09,17630.0,-17540.0,83237.0,93.17,37.0,M-SC,2.35,253.0,-0.99,0.62,9.36,OX40N,NTT,DURABLES
66,SIS,528.00,-0.66,-25.37,62.49,21.26,51780.0,-28170.0,82862.0,1953.37,43.0,H-SC,4.61,166.0,-0.54,0.62,11.96,OX40N,NTT,MISC
47,KANSAINER,340.00,-1.17,-25.91,53.15,13.47,106194.0,-69867.0,199800.0,-69.48,29.0,H-SC,4.21,158.0,-0.66,1.49,1.59,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.55,6.39,61.91,72.27,111613.0,10835.0,180283.0,-8.8,67.0,M-LC,1.93,99.0,0.1,1.34,12.96,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.55,6.39,61.91,72.27,111613.0,10835.0,180283.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.34,12.96,XR,NTT,IT
38,INDIAMART,4810.62,0.13,-3.85,114.99,106.72,136370.0,-4743.0,118593.0,-52.34,31.0,H-SC,1.99,139.0,-0.03,0.88,18.63,AR,ATH,MISC
25,FINCABLES,1641.55,-1.13,-7.26,124.18,107.91,170041.0,-10713.0,136931.0,-24.60,37.0,M-SC,12.38,216.0,-0.06,1.02,2.10,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.16,-2.76,39.48,35.63,80454.0,-5790.0,203784.0,-16.30,37.0,H-MC,3.16,119.0,-0.07,1.52,13.45,AR,ATH,PHARMA
37,IEX,219.00,1.50,-3.93,54.70,48.63,105077.0,-7857.0,192097.0,-36.00,49.0,H-SC,14.06,136.0,-0.07,1.43,7.17,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,2.72,-18.52,49.22,21.58,56201.0,-25956.0,114183.0,-58.94,24.0,X-MC,7.16,56.0,-0.46,0.85,2.72,X40N,ATH,IT
75,TMPV,600.00,-0.04,-18.66,74.80,42.18,168172.0,-51581.0,224829.0,-26.74,24.0,X-LC,6.09,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.34,-21.56,89.26,48.45,140385.0,-43230.0,157276.0,-27.18,25.0,X-MC,12.19,64.0,-0.31,1.17,2.40,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-0.07,-10.53,27.04,13.67,62192.0,-27063.0,230000.0,-17.28,28.0,X-LC,2.63,14.0,-0.44,1.71,8.58,XY25,NTT,FMCG
51,MEDANTA,1486.00,-0.62,-3.99,29.24,24.08,35637.0,-5071.0,121879.0,-11.75,32.0,X-SC,5.40,89.0,-0.14,0.91,14.84,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,1.49,-80.39,536.46,24.79,264909.0,-202475.0,49381.0,-81.32,41.0,X-SC,1.22,86.0,-0.76,0.37,20.72,X40N,NTT,MISC
8,AWL,485.0,0.45,-21.43,95.37,53.50,226102.0,-64672.0,237079.0,-61.97,34.0,X-MC,1.70,58.0,-0.29,1.77,5.30,XY24,NTT,FMCG
43,ITC,452.0,-0.22,-1.72,12.37,10.44,24332.0,-3438.0,196700.0,-41.15,45.0,X-LC,1.76,5.0,-0.14,1.47,3.75,X40,NTT,FMCG
36,ICICIPRULI,790.0,-1.12,5.38,24.28,30.97,45220.0,9507.0,186245.0,-19.43,59.0,X-MC,2.02,75.0,0.21,1.39,18.62,X40,ATH,INSURANCE
3,ACC,3906.0,-0.20,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.20,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.33,0.00,XY24,BTT,CEMENT
75,TMPV,600.00,-0.04,-18.66,74.80,42.18,168172.0,-51581.0,224829.0,-26.74,24.0,X-LC,6.09,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,0.00,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,34.0,X-MC,5.22,57.0,-0.15,1.41,0.10,X40,ATH,FMCG
31,HAVELLS,2069.16,0.18,-10.85,48.60,32.48,142108.0,-35587.0,292404.0,-15.33,35.0,X-MC,5.52,67.0,-0.25,2.18,0.18,X40,ATH,ELECTRICAL
53,PAGEIND,51605.08,0.83,-9.61,39.47,26.07,58416.0,-15740.0,148000.0,-33.39,32.0,X-MC,12.14,55.0,-0.27,1.10,0.83,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.02,-29.80,103.46,42.84,47216.0,-19369.0,45637.0,-54.27,41.0,X-SC,36.53,83.0,-0.41,0.34,3.25,XY24,NTT,MISC
15,CAMS,4762.00,1.49,-80.39,536.46,24.79,264909.0,-202475.0,49381.0,-81.32,41.0,X-SC,1.22,86.0,-0.76,0.37,20.72,X40N,NTT,MISC
59,RELAXO,1176.00,-1.47,-47.46,189.91,52.31,144829.0,-68898.0,76262.0,-44.05,43.0,X-SC,6.43,91.0,-0.48,0.57,2.08,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.14,-37.98,118.02,35.20,94175.0,-48874.0,79796.0,4.55,34.0,X-SC,15.67,92.0,-0.52,0.59,1.36,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.72,-18.52,49.22,21.58,56201.0,-25956.0,114183.0,-58.94,24.0,X-MC,7.16,56.0,-0.46,0.85,2.72,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.21,-13.53,37.94,19.28,113499.0,-46801.0,299155.0,-24.60,58.0,X-LC,5.22,1.0,-0.41,2.23,10.59,X40,ATH,IT
41,INFY,2275.00,0.04,7.93,43.50,54.87,146895.0,24802.0,337690.0,-16.01,60.0,X-LC,3.21,2.0,0.17,2.52,16.87,X40,BTT,IT
75,TMPV,600.00,-0.04,-18.66,74.80,42.18,168172.0,-51581.0,224829.0,-26.74,24.0,X-LC,6.09,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,-0.17,-4.63,42.25,35.67,127271.0,-14610.0,301232.0,-15.92,53.0,X-LC,4.61,4.0,-0.11,2.25,8.59,X40N,ATH,FMCG
43,ITC,452.00,-0.22,-1.72,12.37,10.44,24332.0,-3438.0,196700.0,-41.15,45.0,X-LC,1.76,5.0,-0.14,1.47,3.75,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.11,-17.88,121.22,81.67,93389.0,-16769.0,77041.0,6922.22,40.0,L-SC,13.84,271.0,-0.18,0.57,49.19,XR,NTT,CERAMICS
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.72,39.54,XR,ATH,FINANCE
13,BSOFT,831.70,1.39,-22.22,93.87,50.79,102689.0,-31258.0,109395.0,-0.27,67.0,H-SC,5.86,151.0,-0.30,0.82,27.37,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.33,-14.02,16.33,0.02,24508.0,-24482.0,150077.0,128.79,48.0,M-SC,16.13,234.0,-1.00,1.12,30.54,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.74,-2.70,24.30,20.95,59527.0,-6802.0,244966.0,-3.72,47.0,X-LC,18.66,31.0,-0.11,1.83,31.97,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.39,-22.22,93.87,50.79,102689.0,-31258.0,109395.0,-0.27,67.0,H-SC,5.86,151.0,-0.30,0.82,27.37,XR,ATH,IT
84,WIPRO,420.00,0.55,6.39,61.91,72.27,111613.0,10835.0,180283.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.34,12.96,XR,NTT,IT
56,RAJESHEXPO,518.00,0.51,-63.06,170.92,0.07,86602.0,-86509.0,50668.0,1757.32,56.0,L-SC,2.37,268.0,-1.00,0.38,26.01,OX40N,NTT,JEWELLERY
32,HCLTECH,1922.01,-0.25,7.85,15.65,24.73,40834.0,18994.0,260918.0,12.76,62.0,X-LC,6.18,7.0,0.47,1.94,24.28,X40,ATH,IT
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.72,39.54,XR,ATH,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.75
2,50,76.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.30
LC    24.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.28
X40      22.45
X40N     10.83
XR       10.15
XY25      9.40
AR        9.38
OX40N     7.77
SR        1.01
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.57
X-MC    23.53
X-LC    21.07
M-SC    12.11
X-SC     6.97
H-MC     5.01
L-SC     1.41
M-MC     1.40
M-LC     1.34
H-LC     1.18
L-LC     1.06
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.43,-7.66,42.47
IT,12.75,-17.84,81.54
FINANCE,9.58,-20.58,71.53
ELECTRICAL,6.24,-12.25,53.36
MISC,6.10,-58.53,123.47
PAINTS,6.06,-13.63,30.51
INSURANCE,4.64,-1.72,37.57
PHARMA,4.11,-1.16,33.54
AUTO,3.42,-25.42,79.20


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241625.0,21
AR,1315284.0,10
XR,1301518.0,13
X40,1026202.0,14
X40N,996961.0,9
OX40N,750535.0,10
XY25,378343.0,6
SR,284044.0,2
MH,84748.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3684324.0,25
M-SC,1432819.0,15
X-MC,1324994.0,16
X-SC,985677.0,8
X-LC,936486.0,11
H-MC,407224.0,3
L-SC,271576.0,3
M-LC,111613.0,1
H-LC,75327.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1289178.0      6
           AR         902167.0      5
M-SC       XY24       816270.0      6
H-SC       XR         775468.0      7
X-SC       X40N       507961.0      3
X-MC       X40        458985.0      7
           XY24       397537.0      3
X-LC       X40        385087.0      5
H-SC       OX40N      348719.0      4
M-SC       OX40N      315214.0      5
X-SC       XY24       295586.0      3
H-SC       SR         284044.0      2
X-MC       X40N       283752.0      4
X-LC       XY24       246813.0      2
H-MC       AR         210983.0      2
X-LC       X40N       205248.0      2
H-MC       XY24       196241.0      1
L-SC       XR         184974.0      2
X-MC       XY25       184720.0      2
X-SC       X40        182130.0      2
M-SC       XR         174528.0      2
           AR         126807.0      2
M-LC       XR         111613.0      1
X-LC       XY25        99338.0      2
L-SC       OX40N       86602.0      1
H-SC       MH          84748.0      1
H-LC       AR          75327.0      1
M-MC       XY25        57201.0      1
L-MC       XR          54935.0      1
L-LC       XY25        37084.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
